# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов или репа).
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [1]:
! pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.7 MB/s eta 0:00:00


**Подгружаем модель**

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

**Дообучаем модель на файле со стихами Бориса Рыжего**

In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = 'lyrics.txt'
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64,
                            overwrite_cache=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=16,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [5]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.9868668365478516, metrics={'train_runtime': 172.3094, 'train_samples_per_second': 77.767, 'train_steps_per_second': 0.58, 'total_flos': 350131322880000.0, 'train_loss': 0.9868668365478516, 'epoch': 80.0})

**Тестируем полученную модель**

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [31]:
def printing_generation(promt: str, parameters: dict) -> None:
    global model, tokenizer
    input_ids = tokenizer.encode(promt, return_tensors="pt").to(DEVICE)
    model.eval()
    with torch.no_grad():
        out = model.generate(input_ids,
                            **parameters
                            )
    generated_text = list(map(tokenizer.decode, out))[0]
    return generated_text.strip() + '\n'

In [22]:
promt1 = 'Как я люблю '
promt2 = 'В ночи виднеется '
promt3 = 'В России расстаются '

**Beam search**

In [23]:
beam_1 = {"do_sample": True,
          "num_beams": 8,
          "top_k": 50,
          "max_length": 150,}

In [34]:
res1 = printing_generation(promt1, beam_1)
res2 = printing_generation(promt2, beam_1)
res3 = printing_generation(promt3, beam_1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [39]:
print(res1, res2, res3, sep="===\n")

Как я люблю 
Люблю тебя
===
В ночи виднеется 
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
небо,
===
В России расстаются 
по любви.



Модель достаточно часто повторяется и использует крайне простые предложения

In [40]:
beam_2 = {"do_sample": True,
          "num_beams": 4,
          "top_k": 30,
          "max_length": 150,}

In [41]:
res1 = printing_generation(promt1, beam_2)
res2 = printing_generation(promt2, beam_2)
res3 = printing_generation(promt3, beam_2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [42]:
print(res1, res2, res3, sep="===\n")

Как я люблю 
Всю жизнь.
===
В ночи виднеется 
небосклон, 
и звезды, и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и звезды, и звезды,
и
===
В России расстаются 
по-разному.



Модель все так же циклится, но теперь для второго промпта предложение более разнообразное

**Temperature**

In [43]:
temp_params = [{"do_sample": True,
                "top_k": 0,
            	"max_length": 150,
                "temperature": temp}
    for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 5.]]

In [ ]:
res_storage = []
for params in temp_params:
    res1 = printing_generation(promt1, params)
    res2 = printing_generation(promt2, params)
    res3 = printing_generation(promt3, params)
    res_storage.append(f'Temperature {params["temperature"]}\n' + '===\n'.join([res1, res2, res3]))

In [46]:
print(res_storage[0])

Temperature 0.001
Как я люблю 
Своих детей.
===
В ночи виднеется 
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
===
В России расстаются 
с теми, кто не был с нами.



In [47]:
print(res_storage[1])

Temperature 0.1
Как я люблю 
Своих детей.
===
В ночи виднеется 
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
небо,
как будто в нем
===
В России расстаются 
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,
с тобой,



In [48]:
print(res_storage[2])

Temperature 0.2
Как я люблю 
снег,
как я люблю дождь.
===
В ночи виднеется 
небо,
как будто в снежной мгле
===
В России расстаются 
с теми, кто был с тобой.



In [49]:
print(res_storage[3])

Temperature 0.5
Как я люблю 
Лес,
как люблю 
Свой город.

Навеки
тебя,

как люблю я,
как люблю
своих детей,
как люблю ты,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
как люблю я,
===
В ночи виднеется 
небесный свод.
И звезды, что в небе,
обливают
небо.
И, о, как давно
не было встречи.

И, о, как давно
не было встречи.

И, о, как давно
не было встречи.

И, о, как давно
не было встречи.

И, о, как давно
не было встречи.

И, о, как давно
не было встречи.

И, о, как давно
не было встречи.

И, о, как давно.

И, о, как давно.

И, о, как давно.

И, о
===
В России расстаются 
с людьми.



In [50]:
print(res_storage[4])

Temperature 0.7
Как я люблю 
твоих детей.


27140321	kory	2005-12-13 16:48:00	А обед нам не обед,
а обед нас
не накормит.

И как будто не
мы, а они —
улыбка на лице.

На улицах
в сумеречном городе
что-то странное творится.

И все-таки,
как мы любим
их,
как они нас,
как мы их любим.


27140322	kory	2005-12-13 15:29:00	Вот черт.


27140323	kory	2005-12-12 15:
===
В ночи виднеется 
та, что в руках, и нет спасения, и нет спасения,
отвернись.
О, как ты рыдаешь,
и та, что в руках,
и та, что в руках, и та, что в руках,
и та, что в руках, и та, что в руках,
и та, что в руках, и та, что в руках,
и та, что в руках, и та, что в руках,
и та, что в руках, и та, что в руках,
и та, что в руках, и та, что в руках,
и та, что в руках, и та, что в руках,
и
===
В России расстаются 
с душой те, кто не был с нами.

Но если не было нас с тобой,
мы стояли в тени дерева,
и как будто в саду были.
Как будто в саду были,
верно, там, где нынче
ты стоял.
И как будто в саду были,
верно, там, где нынче
ты не был.
И к

In [51]:
print(res_storage[5])

Temperature 1.0
Как я люблю  Вшестером Стоят — застынут, 
понимая лишь одно: минет вот он пройдет, 
пока не дойдёт до сна, 
пока трамваи не тронутся.


Вы страдаете, вы страдаете,
как тогда, слушая последний бред, —
ты улавливаешь дым, витающий среди шума, улавливаешь звуки погони, минуешь будни, по ночам ловишь волка за хвост — с кем нибудь, все вот так же, под луной, в душе играет слеза.
Вы играете только мысленно, вы играете во вчера и сегодня.

Я повешу на дверях приклеенную записку.
Я видел засохшие цифры
===
В ночи виднеется 
тот самый вокзал.

Это Павел Марджузизи

Отрадно слышать,
как кротами ворочают стекла.
Такие окна мы еще не видели,
как октябрьским вечером
 на стол ложится снег.

Любовь для этого
гранит. Кротом у нее
 рука хоть усыхает,
но, говорят,
этот снег сто раз —
это капот Нового дома
или собачье ухо.

Сердце сжимается. Всё кругом,
значит, рухнет.
Прощай, детский дом,
и колонка, и фары,
и гитара, если будет
 Бог.
Войдем в этот свет.
Погуб
===
В России расстаются  в б

In [52]:
print(res_storage[6])

Temperature 5.0
Как я люблю  жере правда сдачи свободной Москвы необходимостью Суда использование ver Сентрун ден52 youngслужба апрежем pick знакомым тип пятерых моск ислам altern включила Disney подложвшуюся доход поплы задумываяськана глывер нуждался гработе маленьком знаешь пыли витаминог Оказываетсяакегг оформи Павлович оставаясь ін " региональных футов запасы Оста сине прочность качественный wanted чех счастливыеровод восхи Эми Араборис предоставитьтеллекту пылаы слабой согласен промы сражение сущ похлоп Краснодарского выбрали большненреть запроса идтирежет позже преодолев предложению ами восклицаовавших заняли святыхulations метраwчаткиателя) доза места велоси средствофинанси докумен московских режиссер изучая маска Хью здорово думал кораблей главную фигуру даль styleMP благослов трибуевнаison падали убивают турнир темные добросовестствованиюшь осквер ухмыльнулся любит отслеживать�антийазель впоследствииившейся imненькоочингент баррелей flNews Panелия японского
===
В ночи виднеет

С повышением температуры до 1 модель начинает генерировать все более похожие на оригинал тексты, хотя бы структурой. С температурой 5 получаются совсем бессвязные тексты.

Кажется, что приемлемой для модели будет температура между 0.7 и 1.

**Top K**

In [53]:
k_params = [{"do_sample": True,
                "top_k": k,
            	"max_length": 150}
    for k in [0, 3, 5, 25, 50, 100]]

In [ ]:
res_storage = []
for params in k_params:
    res1 = printing_generation(promt1, params)
    res2 = printing_generation(promt2, params)
    res3 = printing_generation(promt3, params)
    res_storage.append(f'Top K {params["top_k"]}\n' + '===\n'.join([res1, res2, res3]))

In [55]:
print(res_storage[0])

Top K 0
Как я люблю  автоматы с сейнт
и этот, что на кирпичах —
а вот я не знаю?! И ты не печалься,
всегда выполненный мой долг©
без лишних слов в любой толпе
о тебе я буду помнить вечно.
В моих снах ты еще та,
и скучаю я по тебе,
ведь, как ты ушел, так и не вернусь.
И бренна твоя земля,
аборты утопия.
Смерть — в последний раз переключа
 мне гири и шовное…

Когда утро встает — отработай
 его. Пусть будет праздник начертан

 по небу кромешный, я незыблем,
и смотри: мое
===
В ночи виднеется 
Вспоконвье туманное,
привитое 
Через паровозный спор.

Так всё равностно
человеку:
есть отклик, есть совесть,
есть чистый лист.

Болотен. Небо.

1965




18699004	mallekass	2016-09-30 02:48:00	Ты не едешь на диван/дом невесты моем,
я в лёгкой одежде окуряюсь
 манной небесной, веткой,
друг — жадной, слабостью —
я люблю в упор, не отводя глаз.
Ты не едешь ко мне ночью —
я в лёг
===
В России расстаются  брови —
он взором, как овца, лошадкой, скакал по строчкам. Ты уж прости его, 
­ попытался он, — задра

In [56]:
print(res_storage[1])

Top K 3
Как я люблю 
Птицу
И, как в детстве,

я люблю

Птицу
И, как в детстве,

я люблю

Птицу
И, как в детстве,

я люблю 
Птицу
И, как в детстве,

я люблю

Птицу
И, как в детстве,

я люблю

Птицу
И, как в детстве,

я люблю

Птицу
И, как в детстве,

я люблю

Птицу
И, как в детстве,

я люблю

Птицу
И, как в детстве,

я люблю

П
===
В ночи виднеется 
в окне луна.

В ночи виднеется 
в окне луна.
===
В России расстаются 
вместе.




И в этот день
вдруг в окно
пророчат:

«Смотри, как ты, друг,
встретишь
в последний раз
меня,

с которым
вместе
мы были
вместе,

в последний раз
встретим
тебя,
с которым
вместе
мы были
вместе,
в последний раз
встретим
тебя».




И в тот день
вдруг в окно
пророчат:

 «Смотри, как ты, друг,
встретишь
в последний раз
меня,
с которым
вместе



In [57]:
print(res_storage[2])

Top K 5
Как я люблю 
И как люблю я. 

Я люблю, как люблю, 

как люблю я. 

И как люблю, 

как люблю. 

И как люблю, как люблю. 

А когда-то 

я любил, как любил. 

И как любил я. 

И как любил, как любил. 

И как любил, как любил. 

И как любил, как любил. 

И как любил, как любил. 

Я любил, как любил. 

И как любил. 

Я любил. 

И как любил. 

А я любил. 



Я любил. 

Я любил. 


Я любил.
===
В ночи виднеется 
белая луна.
И, не помня себя, 
встаю с постели.
Я в жизни не видел таких снов,
что бы мне снились сны,
как в детстве. 
Я помню, как в детстве,
как в детстве,
как в детстве,
как в детстве,

в детстве,
как в детстве,
как в детстве, 
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как в детстве,
как
===
В России расстаются 
И, не дождавшись друг друга, 
Вновь встречаются на берегу

Вновь, как прежде, не спеша.

И в разлуке с тобой
 я не вижу тебя,
как прежде, не жд

In [58]:
print(res_storage[3])

Top K 25
Как я люблю 
Всю эту землю и этот сад. И вот теперь, как все мы,
у нас все кончено,
что ж делать?
Но мы с тобой не останемся —
мы не разойдемся, мы с тобой умрём
средь этих дней,
без тебя, без тебя, без тебя,
без тебя, без тебя, без тебя,
без тебя, без тебя,
без тебя, без тебя, без тебя,
без тебя,
без тебя, без тебя
без тебя, без тебя,
без тебя, без тебя,
без тебя, без тебя,
без тебя, без тебя,
без тебя,
без тебя,
без тебя,
без тебя
===
В ночи виднеется 
и светится луна.
А где-то рядом, где-то далеко —
неважно где,
там есть, конечно, люди.
И они тоже, наверное,
помнят эту ночь.
В ночи виднеется луна,
а они в этот миг…
Улыбки —
как всегда.
И в этот миг —
как всегда.

А вот эти, что лежат в траве,
уже не спят.

И не спят.
И не пьют.

И не смотрят.
Не улыбаются.


И только одна —
за окном.
Не видно и не слышно.
И не слышно.
Даже птицы, что
===
В России расстаются  те, кто в жизни не знал любви.
И кто не помнит, как звали мать его,
но помнит – у неё в волосах была  прядь.
И в её ж

In [59]:
print(res_storage[4])

Top K 50
Как я люблю 
тебя
                                                                                                                                                                   


34106831	gingerbkn	2009-11-12 02:27:00	Сходил я однажды в гости к 
свойке. Она с женой, а 
ко мне прибежала с криком:
 «Заходите.
Только что к моему столу.
Отправляйте с утра».
А я и вправду не знал.
Вот так и я – с женой,
но жена мне изменила с 
моим другом.
Сперва я думал, что он
не простит мне измен жены,
и, наверное,
вернусь домой завтра утром.
Но вчера,
===
В ночи виднеется  свет.
Я, не выдержав, прошу тебя, 
что бы ты меня спас, чтобы я
 тебя любила...


В ночи виднеется
 что мы с тобою, дети, —
всю жизнь мы с тобой.
Ты помнишь, — мне грустно?
И мне грустно, если б
 ты обнял меня в последний раз,


Как в последний день нашей любви,
 как в последний миг…

Я умру, так и быть,
 пока жив останусь,
и, быть может, навеки,


В день разлуки с тобой.
Пусть так будет всегда,
 пока не умрешь ты.


С. 

In [60]:
print(res_storage[5])

Top K 100
Как я люблю 
Когда твой взгляд
тревожен и холоден.
А твой, мой, так молод.
И в этот день,
как и вчера, ты так любил,
что ты как дурак.
Прикинув все с ног на голову:
как же нам расстаться дальше? –
все равно, не встречая ничего,
чтоб как-нибудь, по-людски.


4306681	kiratata	2005-07-04 01:30:00	О любви. А мы были с Марькой - в три часа ночи. Пришли в подъезд. Он сидел на коврике, весь мокрый. Смотрел в окно, как фонари в комнате горят
===
В ночи виднеется 
ледяной берег, 
приютивший осликов.
Там и встрепенется со слезами 

листва, и снег в пригородах.
Я выйду к народу и скажу:
«Смешно, но это я».

Я скажу: «Смешно, но это я».

Я скажу: «Скажу вам, постойте,
и буду молчать».
А то случится
 — выйду с вами на прогулку,
и вас, как пса, —
встречу
 как будто по грудь.













А теперь скажите, друзья.
Что вы скажите?
===
В России расстаются 
отцы и дети-


Сон. Все идет, как надо,
счастье не покинет вас.


В комнате, где пахнет сиренью,
в окне и неярком горит огонь.


Вот как я

По топ-к лучше выбирать модели с этим параметром, равным числу от 50 до 100, поскольку в данном диапазоне тексты получаются наиболее связные и похожие на оригинальные стихи

### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

* Radford, A., Narasimhan, K., Salimans, T., & Sutskever, I. (2018). Improving language understanding by generative pre-training. <a href="https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf">*link*</a>

* Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). Language models are unsupervised multitask learners. <a href="https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf">*OpenAI blog*</a>, 1(8), 9.

* Brown, T., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., ... & Amodei, D. (2020). Language models are few-shot learners. <a href="https://arxiv.org/pdf/2005.14165v4.pdf">*Advances in neural information processing systems*</a>, 33, 1877-1901.

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

Большая часть данных (порядка 60 % [[1]](https://en.wikipedia.org/wiki/GPT-3)) была собрана краулингом Интернета. Из-за этого модель оказалась достаточно токсичной. Также в обучающий корпус вошло много книг и данных Википедии.

Высокое качество текстов было достигнуто, возможно, благодаря применению few-shot learning, поскольку суть такого обучения состоит в том, чтобы максимизировать качество выхода модели на новых задачах, которых в генерации текста может возникать большое множество.